In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name
username = "aacuser"
password = "SNHU1234"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invalid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will return a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'], inplace=True)

## Debug
print(len(df.to_dict(orient='records')))
print(df.columns)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()

# FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
# FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display': 'none'}),
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Center(html.B(html.H1('Kendall Ferebee Project 2 Dashboard'))),
    html.Hr(),
    html.Div(
        html.Img(id='customer-image', src='data:image/png;base64,{}'.format(encoded_image), alt='customer image'), 
             style={'textAlign':'center'}
            ),
    html.Div(
        # FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'mount'},
                {'label': 'Disaster Rescue and Individual Tracking', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='reset'
        )
    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        # FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        # If you completed the Module Six Assignment, you can copy in the code you created here 
        page_size=10,  # Limit number of rows displayed
        style_table={'height': '300px', 'overflowY': 'auto'},
        row_selectable='single',  # Enable single-row selection
        sort_action='native',  # Enable sorting
        filter_action='native',  # Enable filtering
        selected_rows=[0],  # Default selection
    ),
    html.Br(),
    html.Hr(),
    # This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

#############################################
# Interaction Between Components / Controller
############################################# 
@app.callback(
    Output('datatable-id', 'data'),
    Output('datatable-id', 'columns'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):   
    # FIX ME Add code to filter interactive data table with MongoDB queries
    query = {}
    if filter_type == 'water':
        query = {
            "animal_type": "Dog",
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26.0, "$lte": 156.0}
        }
    elif filter_type == 'mount':
         query = {
            "animal_type": "Dog",
            "breed": {"$in": ["German Shepard", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26.0, "$lte": 156.0}
        }
    elif filter_type == 'disaster':
         query = {
            "animal_type": "Dog",
            "breed": {"$in": ["Doberman Pinscher", "German Shepard", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20.0, "$lte": 300.0}
        }
    
    df = pd.DataFrame.from_records(db.read(query))
    
    if '_id' in df.columns:
        df.drop(columns=['_id'], inplace =True)
        
    if df.empty:
        return [], [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        
    data = df.to_dict('records')
    columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    
    print(f"Data:{data[:5]}")
    print(f"Columns: {columns}")
    
    return data, columns
  
    
# Change background color of selected columns
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is None:
        return []
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    # FIX ME add code for chart of your choice (e.g. pie chart)
    if viewData is None or len(viewData) == 0:
        return dcc.Graph(figure=px.pie(title="No Data Available"))
        
    dff = pd.DataFrame(viewData)
    
    if dff.empty or 'breed' not in dff.columns:
        return dcc.Graph(figure=px.pie(title='No Data Available'))
                
    fig = px.pie(dff, names='breed', title='Breed Distribution')
    
    return dcc.Graph(figure=fig)
    
# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable

# Map
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('datatable-id', 'derived_viewport_selected_rows')]
)
def update_map(viewData, row_ids):
    if not row_ids or not viewData:
        return [dl.Map(style={'width': '1000px', 'height': '500px'}, center=[0, 0], zoom=10, children=[
            dl.TileLayer(id="base-layer-id")
        ])]
    
    dff = pd.DataFrame.from_dict(viewData)
    if len(row_ids) > 0:
        selected_row = row_ids[-1]  # Use the last selected row ID
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'}, center=[dff.iloc[row_ids[0],13],dff.iloc[row_ids[0],14]], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[dff.iloc[row_ids[0],13],dff.iloc[row_ids[0],14]], children=[
                # parameter num 4 gives the breed of animal
                dl.Tooltip(dff.iloc[row_ids[0],4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    # parameter 9 gives animal's name
                    html.P(dff.iloc[row_ids[0],9])
                ])
            ])
        ])
    ]


app.run_server(debug=True)


Connection Successful!
59604
Index(['rec_num', 'age_upon_outcome', 'animal_id', 'animal_type', 'breed',
       'color', 'date_of_birth', 'datetime', 'monthyear', 'name',
       'outcome_subtype', 'outcome_type', 'sex_upon_outcome', 'location_lat',
       'location_long', 'age_upon_outcome_in_weeks'],
      dtype='object')
Dash app running on http://127.0.0.1:16714/
Data:[{'rec_num': 4, 'age_upon_outcome': '7 months', 'animal_id': 'A733653', 'animal_type': 'Cat', 'breed': 'Siamese Mix', 'color': 'Seal Point', 'date_of_birth': '2016-01-25', 'datetime': '2016-08-27 18:11:00', 'monthyear': '2016-08-27T18:11:00', 'name': 'Kitty', 'outcome_subtype': '', 'outcome_type': 'Adoption', 'sex_upon_outcome': 'Intact Female', 'location_lat': 30.3188063374257, 'location_long': -97.7240376703891, 'age_upon_outcome_in_weeks': 30.8225198412698}, {'rec_num': 5, 'age_upon_outcome': '2 years', 'animal_id': 'A691584', 'animal_type': 'Dog', 'breed': 'Labrador Retriever Mix', 'color': 'Tan/White', 'date_of_bir